In [124]:
import pandas as pd
import numpy as np

In [125]:
#Load placeholder dataset until consolidated dataset has been finalized
df_filtered = pd.read_csv('../../Data/filtered/2018.csv')
df_filtered.head()

C:\Users\Kamil\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (4,7,9,10,11,12,13,14,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,country,student,employment_status,formal_education,company_size,occupation,IT_experience_in_years,job_satisfaction,compensation,proficient_languages,database_worked_on,platform_worked_on,framework_worked_on,desktop_OS,work_hours,age,gender,stackoverflow_account_holder,stackoverflow_participation
0,Kenya,No,Employed part-time,"Bachelorâs degree (BA, BS, B.Eng., etc.)",20 to 99 employees,Full-stack developer,3-5 years,Extremely satisfied,NaN,"['JavaScript', 'Python', 'HTML', 'CSS']","['Redis', 'SQL Server', 'MySQL', 'PostgreSQL',...","['AWS', 'Azure', 'Linux', 'Firebase']","['Django', 'React']",Linux-based,9 - 12 hours,25 - 34 years old,Male,Yes,I have never participated in Q&A on Stack Over...
1,United Kingdom,No,Employed full-time,"Bachelorâs degree (BA, BS, B.Eng., etc.)","10,000 or more employees",Database administrator;DevOps specialist;Full-...,18-20 years,Moderately dissatisfied,70841.0,"['JavaScript', 'Python', 'Bash/Shell']","['Redis', 'PostgreSQL', 'Memcached']",['Linux'],['Django'],Linux-based,5 - 8 hours,35 - 44 years old,Male,Yes,A few times per month or weekly
2,United States,No,Employed full-time,Associate degree,20 to 99 employees,Engineering manager;Full-stack developer,6-8 years,Moderately satisfied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,United States,No,Employed full-time,"Bachelorâs degree (BA, BS, B.Eng., etc.)",100 to 499 employees,Full-stack developer,12-14 years,Neither satisfied nor dissatisfied,NaN,"['C#', 'JavaScript', 'SQL', 'TypeScript', 'HTM...","['SQL Server', 'Microsoft Azure (Tables, Cosmo...",['Azure'],NaN,Windows,9 - 12 hours,35 - 44 years old,Male,Yes,A few times per month or weekly
4,South Africa,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"10,000 or more employees",Data or business analyst;Desktop or enterprise...,0-2 years,Slightly satisfied,21426.0,"['C', 'C++', 'Java', 'Matlab', 'R', 'SQL', 'Ba...","['SQL Server', 'PostgreSQL', 'Oracle', 'IBM Db2']","['Arduino', 'Windows Desktop or Server']",NaN,Windows,Over 12 hours,18 - 24 years old,Male,Yes,Less than once per month or monthly


## Steps/comments: ##
- Loading filtered data and dropping NaN and 0 values in 'compensation'.
- Imputation of missing values - TBD; e.g. KNN?
- Encoding of ordinal variables as continuous (e.g. range midpoints) or as dummy variables (e.g. if they have open-ended categories).
- Encoding of categorical variables (e.g. language proficiency) using dummy encoding.
- Gender has multiple responses; use target encoding?
- Decide how to process questions with variable number of responses (e.g. proficient languages).


In [150]:
df_final = pd.DataFrame()

**Dropping NaN and 0 in 'compensation'**

In [151]:
df_salary = df_filtered.dropna(subset=['compensation'])[df_filtered.dropna(subset=['compensation'])['compensation']!=0]
print('Dataframe without NaN and 0 in compensation is {}%'.format(round(len(df_salary)/len(df_filtered)*100, 2)), 'of the original dataframe.')
print('There are {}'.format(len(df_salary)), 'data points in it compared to {}'.format(len(df_filtered)), 'entries in the original dataframe.\n')
print('Information on missing values in the dataframe:')
df_salary.isna().sum()/len(df_salary)*100

Dataframe without NaN and 0 in compensation is 47.4% of the original dataframe.
There are 46860 data points in it compared to 98855 entries in the original dataframe.

Information on missing values in the dataframe:


country                          0.000000
student                          0.802390
employment_status                0.277422
formal_education                 1.410585
company_size                    12.831840
occupation                       0.706359
IT_experience_in_years           1.207853
job_satisfaction                 3.988476
compensation                     0.000000
proficient_languages             2.494665
database_worked_on              14.526248
platform_worked_on              15.228340
framework_worked_on             31.308152
desktop_OS                       3.085787
work_hours                       3.551003
age                              5.887751
gender                           6.545028
stackoverflow_account_holder     2.554417
stackoverflow_participation     14.007682
dtype: float64

**Imputation - TBD**

**Ordinal -> continuous**

In [152]:
#ordinal for continuous encoding:
ord_cont = ['formal_education', 'IT_experience_in_years', 'job satisfaction', 'work_hours', 'age']

In [153]:
#df_salary['formal_education'].unique()
#Transform those into:
# - no formal education 0
# - primary/elementary school 1
# - secondary school 2
# - some college or associate degree 3
# - Bachelor's 4
# - Master's or professional degree (JD/MD) 5
# - Other doctoral degree 6

df_edu = df_salary.copy()
cols_edu = [x for x in df_salary['formal_education'].unique() if x==x]
edu_vals = [4, 3, 5, 3, 2, 5, 6, 1, 0]
edu_mapping = dict(zip(cols_edu, edu_vals))
df_final['formal_education'] = df_salary['formal_education'].replace(edu_mapping)

**Ordinal -> dummy encoding**

In [154]:
ord_dummy = ['company_size']

**Encoding of categorical variables**

In [155]:
cat_dummy = ['student', 'employment_status', 'desktop_OS', 'stackoverflow_account_holder', 'stackoverflow_participation']

In [156]:
#For each column in cat_dummy, create dummy encoding
for col in cat_dummy:
    dummies = pd.get_dummies(df_salary, prefix=col, columns=[col], drop_first=True)
    cols = len([x for x in df_salary[col].unique() if x==x])-1 #Find columns of interest, ignore NaNs at this stage
    #Add to final dataframe
    df_final[dummies.columns[-cols:]] = dummies.iloc[:, -cols:]

df_final.head()
    
#For now do not include 'country'; to be replaced with other data to adjust for region, e.g. GDP?

,formal_education,"student_Yes, full-time","student_Yes, part-time",employment_status_Employed part-time,"employment_status_Independent contractor, freelancer, or self-employed","employment_status_Not employed, and not looking for work","employment_status_Not employed, but looking for work",employment_status_Retired,desktop_OS_Linux-based,desktop_OS_MacOS,desktop_OS_Windows,stackoverflow_account_holder_No,stackoverflow_account_holder_Yes,stackoverflow_participation_A few times per week,stackoverflow_participation_Daily or almost daily,stackoverflow_participation_I have never participated in Q&A on Stack Overflow,stackoverflow_participation_Less than once per month or monthly,stackoverflow_participation_Multiple times per day
1,4.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
4,3.0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0
5,4.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
6,3.0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0
8,3.0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0


In [138]:
#Use target encoding for gender?
df_salary['gender'].unique()

array(['Male',
       'Female;Male;Transgender;Non-binary, genderqueer, or gender non-conforming',
       'Female;Male', 'Female',
       'Male;Non-binary, genderqueer, or gender non-conforming', nan,
       'Non-binary, genderqueer, or gender non-conforming', 'Transgender',
       'Female;Transgender',
       'Transgender;Non-binary, genderqueer, or gender non-conforming',
       'Female;Non-binary, genderqueer, or gender non-conforming',
       'Female;Transgender;Non-binary, genderqueer, or gender non-conforming',
       'Male;Transgender',
       'Female;Male;Non-binary, genderqueer, or gender non-conforming',
       'Male;Transgender;Non-binary, genderqueer, or gender non-conforming',
       'Female;Male;Transgender'], dtype=object)

In [ ]:
#Finding category values for those with variable number of options - TBC
['occupation', 'proficient_languages', 'database_worked_on', 'platform_worked_on', 'framework_worked_on']